# DMT Model: Small Amplitude Oscillatory Shear (SAOS)

> **Handbook:** See [DMT SAOS Protocol](../../docs/source/models/dmt/dmt.rst#saos-small-amplitude-oscillatory-shear) for linear viscoelastic theory and Maxwell moduli derivations.

## Physical Context

In **Small Amplitude Oscillatory Shear (SAOS)**, the applied strain $\gamma(t) = \gamma_0 \sin(\omega t)$ is so small that the structure parameter $\lambda$ remains essentially constant at its initial value $\lambda_0$. This produces **linear viscoelastic response**.

### Maxwell Element at Fixed Structure

At fixed $\lambda_0$, the DMT model (with elasticity) behaves as a **single Maxwell element** with:

- **Elastic modulus**: $G = G(\lambda_0) = G_0 \lambda_0^{m_G}$
- **Viscosity**: $\eta = \eta(\lambda_0)$ from closure (exponential or HB)
- **Relaxation time**: $\theta_1 = \eta/G$

The SAOS moduli are:

$$G'(\omega) = G \frac{(\omega\theta_1)^2}{1 + (\omega\theta_1)^2}$$

$$G''(\omega) = G \frac{\omega\theta_1}{1 + (\omega\theta_1)^2} + \eta_\infty \omega$$

### Crossover Frequency

The crossover frequency $\omega_c$ (where $G' = G''$) occurs at:

$$\omega_c \approx \frac{G}{\eta} = \frac{1}{\theta_1}$$

This provides the characteristic relaxation time of the material.

### Identifiability Limits

From SAOS data alone at fixed $\lambda_0$, we can only identify **3 parameters**:

1. $G_0$ (plateau modulus)
2. $\eta_0$ (zero-shear viscosity)
3. $\eta_\infty$ (infinite-shear viscosity)

The thixotropic parameters $(a, c, t_{\text{eq}}, m_G)$ are **not identifiable** from SAOS alone because the structure doesn't evolve. We need transient tests (startup, creep) to probe structure evolution.

### Effect of Preshear State

The SAOS response depends on initial structure $\lambda_0$:

- Fully structured ($\lambda_0 = 1$): Maximum moduli, lowest crossover frequency
- Partially broken ($\lambda_0 < 1$): Lower moduli, higher crossover frequency

This history-dependence distinguishes thixotropic materials from purely viscoelastic ones.

### Industrial Relevance

- **Quality control**: SAOS provides rapid material fingerprinting
- **Formulation**: Crossover frequency indicates processing windows
- **Stability**: Plateau modulus correlates with gel strength

## Learning Objectives

- Understand linear viscoelastic response in the DMT framework
- Analyze $G'/G''$ crossover and its connection to Maxwell relaxation time
- Recognize that at fixed structure ($\lambda_0$), DMT reduces to a single Maxwell element
- Explore identifiability limits in SAOS (only 3 parameters recoverable)
- Visualize the effect of preshear state ($\lambda_0$) on linear response

## Prerequisites

- Notebook 01 (Flow Curves)
- Understanding of Maxwell model and linear viscoelasticity

## Runtime

- Data generation + NLSQ: ~1 minute
- Bayesian inference (4 chains, 1000+2000 samples): ~2-3 minutes
- Total: ~3-4 minutes

## 1. Setup

### Google Colab Detection and Installation

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running in local environment")

if IN_COLAB:
    print("Installing RheoJAX...")
    !pip install -q rheojax
    
    import os
    os.environ['JAX_ENABLE_X64'] = '1'
    print("JAX float64 enabled")

### Imports

In [ ]:
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from scipy.optimize import curve_fit

%matplotlib inline

# RheoJAX imports with safe JAX configuration
from rheojax.core.jax_config import safe_import_jax, verify_float64

jax, jnp = safe_import_jax()

# Bayesian tools
import arviz as az

from rheojax.core.data import RheoData
from rheojax.models import DMTLocal
from rheojax.models.dmt._kernels import (
    elastic_modulus,
    saos_moduli_maxwell,
    viscosity_exponential,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Verify float64 precision
verify_float64()
print(f"JAX devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

## 2. Theory: SAOS in the DMT Framework

### Linear Viscoelasticity at Fixed Structure

In Small Amplitude Oscillatory Shear (SAOS), the applied strain is so small that the structure parameter λ remains essentially constant at its initial value λ₀. This means:

- **No structure evolution**: dλ/dt ≈ 0 (small amplitude → negligible rejuvenation)
- **Linear response**: Stress is linearly proportional to strain

### Maxwell Element Analogy

At fixed λ₀, the DMT model (with elasticity) behaves as a **single Maxwell element** with:

- **Elastic modulus**: G = G(λ₀) = G₀(λ₀)^(m_G)
- **Viscosity**: η = η(λ₀) from closure (exponential or HB)
- **Relaxation time**: θ₁ = η/G

### SAOS Moduli

For a Maxwell element subjected to γ(t) = γ₀sin(ωt):

$$G'(\omega) = G \frac{(\omega\theta_1)^2}{1 + (\omega\theta_1)^2}$$

$$G''(\omega) = G \frac{\omega\theta_1}{1 + (\omega\theta_1)^2} + \eta_\infty \omega$$

Where:
- The first term in G'' is the Maxwell contribution
- η∞·ω is the Newtonian solvent contribution

### Special Case: Fully Built Structure (λ₀ = 1)

At λ₀ = 1:
- G = G₀ (maximum elasticity)
- η = η₀ (maximum viscosity)
- θ₁ = η₀/G₀

This is the equilibrium state of an unsheared material.

### Identifiability

From SAOS data alone at fixed λ₀, we can only identify:
1. **G₀** (plateau modulus)
2. **η₀** (zero-shear viscosity)
3. **η∞** (infinite-shear viscosity)

The thixotropic parameters (a, c, t_eq, m_G) are **not identifiable** from SAOS alone because the structure doesn't evolve. We need transient tests (startup, creep) to probe structure evolution.

## 3. Generate Synthetic SAOS Data

We'll create synthetic data from a calibrated DMT model with known parameters.

In [ ]:
# Calibrated parameters (from previous notebooks)
calib_params = {
    "eta_0": 1.5e4,    # Pa·s (zero-shear viscosity)
    "eta_inf": 0.3,    # Pa·s (infinite-shear viscosity)
    "a": 0.8,          # Rejuvenation rate coefficient
    "c": 0.7,          # Shear-rate exponent
    "G0": 500.0,       # Pa (elastic modulus at λ=1)
    "m_G": 1.0,        # Structure exponent for G
    "t_eq": 50.0,      # s (equilibration time)
}

# Create model and set parameters
model_true = DMTLocal(closure="exponential", include_elasticity=True)
for param_name, value in calib_params.items():
    if param_name in model_true.parameters:
        model_true.parameters[param_name].value = value

# Generate frequency sweep
omega = np.logspace(-2, 3, 50)  # 0.01 to 1000 rad/s

# Predict SAOS at fully built structure (λ₀ = 1)
G_prime_true, G_double_prime_true = model_true.predict_saos(omega, lam_0=1.0)

# Add 3% Gaussian noise
np.random.seed(42)
noise_level = 0.03
G_prime_noisy = G_prime_true * (1 + noise_level * np.random.randn(len(omega)))
G_double_prime_noisy = G_double_prime_true * (1 + noise_level * np.random.randn(len(omega)))

# Combine into complex modulus
G_star = G_prime_noisy + 1j * G_double_prime_noisy

print(f"Generated {len(omega)} frequency points")
print(f"Frequency range: {omega[0]:.2e} - {omega[-1]:.2e} rad/s")
print(f"Noise level: {noise_level*100}%")

### Visualize Synthetic Data

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plot G' and G''
ax.loglog(omega, G_prime_noisy, 'o', label="G' (Storage)", markersize=6, alpha=0.7)
ax.loglog(omega, G_double_prime_noisy, 's', label="G'' (Loss)", markersize=6, alpha=0.7)

# Plot noise-free truth (thin lines)
ax.loglog(omega, G_prime_true, '-', color='C0', linewidth=1, alpha=0.5, label="G' (true)")
ax.loglog(omega, G_double_prime_true, '-', color='C1', linewidth=1, alpha=0.5, label="G'' (true)")

# Mark crossover frequency
crossover_idx = np.argmin(np.abs(G_prime_true - G_double_prime_true))
omega_c = omega[crossover_idx]
ax.axvline(omega_c, color='gray', linestyle='--', alpha=0.5, label=f'Crossover: {omega_c:.2f} rad/s')

ax.set_xlabel('Angular Frequency ω (rad/s)', fontsize=12)
ax.set_ylabel("G', G'' (Pa)", fontsize=12)
ax.set_title('Synthetic SAOS Data (DMT Model, λ₀ = 1)', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, which='both', alpha=0.3)

display(fig)
plt.close(fig)

print(f"\nCrossover frequency ω_c = {omega_c:.3f} rad/s")
print(f"Expected relaxation time θ₁ = η₀/G₀ = {calib_params['eta_0']/calib_params['G0']:.1f} s")
print(f"Expected ω_c ≈ 1/θ₁ = {calib_params['G0']/calib_params['eta_0']:.4f} rad/s")

## 4. NLSQ Fitting

We'll fit only the identifiable parameters: G₀, η₀, η∞.

Since SAOS doesn't probe structure evolution, we fix the thixotropic parameters at arbitrary values.

In [ ]:
# Define SAOS residual function for fitting
def saos_residual(omega, G0, eta_0, eta_inf):
    """
    Compute SAOS moduli for a Maxwell element.
    
    At λ₀=1: G=G₀, η=η₀, θ₁=η₀/G₀
    """
    omega_jax = jnp.asarray(omega)
    G = G0
    eta = eta_0
    theta_1 = eta / G
    
    # Maxwell element formulas
    omega_theta = omega_jax * theta_1
    G_prime = G * omega_theta**2 / (1 + omega_theta**2)
    G_double_prime = G * omega_theta / (1 + omega_theta**2) + eta_inf * omega_jax
    
    # Return complex modulus
    return G_prime + 1j * G_double_prime

def saos_residual_scipy(omega, G0, eta_0, eta_inf):
    """scipy-compatible wrapper that returns real values (stacked G', G'')."""
    G_star = saos_residual(omega, G0, eta_0, eta_inf)
    # Stack G' and G'' for scipy curve_fit
    return np.concatenate([np.real(G_star), np.imag(G_star)])

# Create model for reference
model_fit = DMTLocal(closure="exponential", include_elasticity=True)

# Perform NLSQ fitting using scipy.optimize.curve_fit
# Note: DMT model.fit() for SAOS is not implemented, so we use scipy directly
print("Running NLSQ optimization with scipy.optimize.curve_fit...")

# Helper function for computing fit quality
def compute_fit_quality(y_true, y_pred):
    """Compute R² and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    # Handle complex numbers
    if np.iscomplexobj(y_true):
        y_true_flat = np.concatenate([np.real(y_true), np.imag(y_true)])
        y_pred_flat = np.concatenate([np.real(y_pred), np.imag(y_pred)])
    else:
        y_true_flat = y_true.ravel() if y_true.ndim > 1 else y_true
        y_pred_flat = y_pred.ravel() if y_pred.ndim > 1 else y_pred
    residuals = y_true_flat - y_pred_flat
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true_flat - np.mean(y_true_flat))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {"R2": r2, "RMSE": rmse}

# Initial guesses
p0 = [300.0, 1e4, 1.0]

# Bounds (lower, upper)
bounds = (
    [10.0, 100.0, 0.01],  # lower
    [5000.0, 1e6, 10.0]   # upper
)

# Stack data for scipy (G' followed by G'')
y_data_stacked = np.concatenate([G_prime_noisy, G_double_prime_noisy])

# Run scipy curve_fit
popt, pcov = curve_fit(
    saos_residual_scipy,
    omega,
    y_data_stacked,
    p0=p0,
    bounds=bounds,
    maxfev=5000
)

# Extract fitted parameters
fitted_params = {
    'G0': popt[0],
    'eta_0': popt[1],
    'eta_inf': popt[2]
}

# Compute metrics
G_star_pred = saos_residual(omega, *popt)
metrics = compute_fit_quality(G_star, G_star_pred)

print("\n" + "="*60)
print("NLSQ Fitting Results")
print("="*60)
print(f"{'Parameter':<15} {'True':<15} {'Fitted':<15} {'Error %':<10}")
print("-"*60)

for param_name in ['G0', 'eta_0', 'eta_inf']:
    true_val = calib_params[param_name]
    fitted_val = fitted_params[param_name]
    error_pct = abs(fitted_val - true_val) / true_val * 100
    print(f"{param_name:<15} {true_val:<15.3e} {fitted_val:<15.3e} {error_pct:<10.2f}")

print("-"*60)
print(f"R² = {metrics['R2']:.6f}")

### Visualize NLSQ Fit

In [ ]:
# Predict with fitted parameters
G_star_fit = saos_residual(omega, 
                           fitted_params['G0'], 
                           fitted_params['eta_0'], 
                           fitted_params['eta_inf'])

G_prime_fit = np.real(G_star_fit)
G_double_prime_fit = np.imag(G_star_fit)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: G' and G'' vs frequency
ax1.loglog(omega, G_prime_noisy, 'o', label="G' (data)", markersize=6, alpha=0.7)
ax1.loglog(omega, G_double_prime_noisy, 's', label="G'' (data)", markersize=6, alpha=0.7)
ax1.loglog(omega, G_prime_fit, '-', color='C0', linewidth=2, label="G' (NLSQ fit)")
ax1.loglog(omega, G_double_prime_fit, '-', color='C1', linewidth=2, label="G'' (NLSQ fit)")

ax1.set_xlabel('Angular Frequency ω (rad/s)', fontsize=12)
ax1.set_ylabel("G', G'' (Pa)", fontsize=12)
ax1.set_title('NLSQ Fit to SAOS Data', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, which='both', alpha=0.3)

# Right: Residuals
residuals_prime = (G_prime_noisy - G_prime_fit) / G_prime_noisy * 100
residuals_double_prime = (G_double_prime_noisy - G_double_prime_fit) / G_double_prime_noisy * 100

ax2.semilogx(omega, residuals_prime, 'o-', label="G' residuals", markersize=5)
ax2.semilogx(omega, residuals_double_prime, 's-', label="G'' residuals", markersize=5)
ax2.axhline(0, color='black', linestyle='--', linewidth=1)
ax2.fill_between(omega, -5, 5, alpha=0.2, color='gray', label='±5%')

ax2.set_xlabel('Angular Frequency ω (rad/s)', fontsize=12)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Fitting Residuals', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, which='both', alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference

We'll perform Bayesian inference to quantify parameter uncertainty. The DMT model's BayesianMixin automatically handles complex-valued data by splitting into real and imaginary parts.

In [ ]:
# Custom Bayesian inference for SAOS using NumPyro
# Note: DMT model.fit_bayesian() for SAOS is not implemented, so we use NumPyro directly
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS


def saos_model(omega, G_prime_obs, G_double_prime_obs):
    """NumPyro model for SAOS fitting."""
    # Priors (use NLSQ as warm-start)
    G0 = numpyro.sample("G0", dist.Uniform(10.0, 5000.0))
    eta_0 = numpyro.sample("eta_0", dist.Uniform(100.0, 1e6))
    eta_inf = numpyro.sample("eta_inf", dist.Uniform(0.01, 10.0))
    
    # Maxwell model prediction
    theta_1 = eta_0 / G0
    omega_theta = omega * theta_1
    G_prime_pred = G0 * omega_theta**2 / (1 + omega_theta**2)
    G_double_prime_pred = G0 * omega_theta / (1 + omega_theta**2) + eta_inf * omega
    
    # Likelihood (Gaussian with estimated noise)
    sigma_G_prime = numpyro.sample("sigma_G_prime", dist.HalfNormal(50.0))
    sigma_G_double_prime = numpyro.sample("sigma_G_double_prime", dist.HalfNormal(50.0))
    
    numpyro.sample("G_prime", dist.Normal(G_prime_pred, sigma_G_prime), obs=G_prime_obs)
    numpyro.sample("G_double_prime", dist.Normal(G_double_prime_pred, sigma_G_double_prime), obs=G_double_prime_obs)

print("Running Bayesian inference (NUTS sampler) with NumPyro...")
print("This may take 1-2 minutes...\n")

# Run NUTS
rng_key = jax.random.PRNGKey(42)
kernel = NUTS(saos_model)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key, omega, jnp.array(G_prime_noisy), jnp.array(G_double_prime_noisy))

# Create result container with to_inference_data() for shared utilities
class BayesResult:
    def __init__(self, samples, num_chains=4):
        self.posterior_samples = samples
        self.num_chains = num_chains
    
    def to_inference_data(self):
        chain_length = len(list(self.posterior_samples.values())[0]) // self.num_chains
        return az.from_dict(
            posterior={
                param: self.posterior_samples[param].reshape(self.num_chains, chain_length)
                for param in self.posterior_samples
            }
        )

bayes_result = BayesResult(mcmc.get_samples(), num_chains=4)

print("\nBayesian inference completed!")

### Diagnostics

In [ ]:
# Compute diagnostics
import arviz as az

print("="*60)
print("Bayesian Diagnostics")
print("="*60)

# Convert to InferenceData for diagnostics
idata = az.from_dict(
    posterior={
        param: bayes_result.posterior_samples[param].reshape(4, -1)
        for param in ['G0', 'eta_0', 'eta_inf']
        if param in bayes_result.posterior_samples
    }
)

# Get summary with R-hat and ESS
summary = az.summary(idata, var_names=['G0', 'eta_0', 'eta_inf'])
print(summary[['mean', 'sd', 'hdi_3%', 'hdi_97%', 'ess_bulk', 'r_hat']])

print("-"*60)
print("R-hat < 1.01: Excellent convergence")
print("ESS > 400: Good effective sample size")

### Trace Plot

In [ ]:
saos_param_names = ['G0', 'eta_0', 'eta_inf']
display_arviz_diagnostics(bayes_result, saos_param_names, fast_mode=FAST_MODE)

### Posterior Predictive Check

In [ ]:
# Sample from posterior for predictions
n_posterior_samples = 200
sample_indices = np.random.choice(len(bayes_result.posterior_samples['G0']), 
                                  size=n_posterior_samples, 
                                  replace=False)

G_prime_samples = []
G_double_prime_samples = []

for idx in sample_indices:
    G0_sample = bayes_result.posterior_samples['G0'][idx]
    eta_0_sample = bayes_result.posterior_samples['eta_0'][idx]
    eta_inf_sample = bayes_result.posterior_samples['eta_inf'][idx]
    
    G_star_sample = saos_residual(omega, G0_sample, eta_0_sample, eta_inf_sample)
    G_prime_samples.append(np.real(G_star_sample))
    G_double_prime_samples.append(np.imag(G_star_sample))

G_prime_samples = np.array(G_prime_samples)
G_double_prime_samples = np.array(G_double_prime_samples)

# Compute credible intervals
G_prime_lower = np.percentile(G_prime_samples, 2.5, axis=0)
G_prime_upper = np.percentile(G_prime_samples, 97.5, axis=0)
G_prime_median = np.percentile(G_prime_samples, 50, axis=0)

G_double_prime_lower = np.percentile(G_double_prime_samples, 2.5, axis=0)
G_double_prime_upper = np.percentile(G_double_prime_samples, 97.5, axis=0)
G_double_prime_median = np.percentile(G_double_prime_samples, 50, axis=0)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

# Data
ax.loglog(omega, G_prime_noisy, 'o', label="G' (data)", markersize=6, alpha=0.7, color='C0')
ax.loglog(omega, G_double_prime_noisy, 's', label="G'' (data)", markersize=6, alpha=0.7, color='C1')

# Posterior median
ax.loglog(omega, G_prime_median, '-', linewidth=2, label="G' (Bayesian median)", color='C0')
ax.loglog(omega, G_double_prime_median, '-', linewidth=2, label="G'' (Bayesian median)", color='C1')

# Credible intervals
ax.fill_between(omega, G_prime_lower, G_prime_upper, alpha=0.3, color='C0', label='95% CI (G\')')
ax.fill_between(omega, G_double_prime_lower, G_double_prime_upper, alpha=0.3, color='C1', label='95% CI (G\'\')')

ax.set_xlabel('Angular Frequency ω (rad/s)', fontsize=12)
ax.set_ylabel("G', G'' (Pa)", fontsize=12)
ax.set_title('Bayesian Posterior Predictive Check', fontsize=14)
ax.legend(fontsize=10, loc='best')
ax.grid(True, which='both', alpha=0.3)

display(fig)
plt.close(fig)

## 6. Effect of Preshear State (λ₀)

The SAOS response depends on the initial structure λ₀. A material with partially broken structure (λ₀ < 1) will have lower moduli and a shifted crossover frequency.

In [ ]:
# Predict SAOS at different structure levels
lambda_values = [0.2, 0.5, 0.8, 1.0]
colors = plt.cm.viridis(np.linspace(0, 1, len(lambda_values)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for lam, color in zip(lambda_values, colors):
    G_prime, G_double_prime = model_true.predict_saos(omega, lam_0=lam)
    
    ax1.loglog(omega, G_prime, '-', linewidth=2, color=color, label=f'λ₀ = {lam}')
    ax1.loglog(omega, G_double_prime, '--', linewidth=2, color=color)
    
    # Find crossover
    crossover_idx = np.argmin(np.abs(np.asarray(G_prime) - np.asarray(G_double_prime)))
    omega_c = omega[crossover_idx]
    ax2.plot(lam, omega_c, 'o', markersize=10, color=color)

ax1.set_xlabel('Angular Frequency ω (rad/s)', fontsize=12)
ax1.set_ylabel("G', G'' (Pa)", fontsize=12)
ax1.set_title('SAOS at Different Structure Levels', fontsize=13)
ax1.legend(fontsize=10, title="Solid: G', Dashed: G''", title_fontsize=9)
ax1.grid(True, which='both', alpha=0.3)

ax2.set_xlabel('Structure Parameter λ₀', fontsize=12)
ax2.set_ylabel('Crossover Frequency ω_c (rad/s)', fontsize=12)
ax2.set_title('Crossover vs Structure Level', fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print("\nObservations:")
print("- Lower λ₀ → Lower moduli (less elastic network)")
print("- Crossover frequency shifts with structure")
print("- SAOS alone cannot distinguish thixotropy from simple viscoelasticity!")

## 7. Cole-Cole Plot

A Cole-Cole plot (G'' vs G') reveals the number of relaxation modes. A single Maxwell element produces a perfect semicircle.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# Plot data
ax.plot(G_prime_noisy, G_double_prime_noisy, 'o', 
        label='Data', markersize=6, alpha=0.7)

# Plot fitted curve
ax.plot(G_prime_fit, G_double_prime_fit, '-', 
        linewidth=2, label='NLSQ Fit', color='C1')

# Theoretical semicircle (no solvent contribution)
G_theory = fitted_params['G0']
theta = np.linspace(0, np.pi, 100)
G_prime_circle = G_theory / 2 * (1 + np.cos(theta))
G_double_prime_circle = G_theory / 2 * np.sin(theta)
ax.plot(G_prime_circle, G_double_prime_circle, '--', 
        linewidth=1.5, label='Perfect Maxwell (no solvent)', color='gray', alpha=0.5)

ax.set_xlabel("G' (Pa)", fontsize=12)
ax.set_ylabel("G'' (Pa)", fontsize=12)
ax.set_title('Cole-Cole Plot', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

display(fig)
plt.close(fig)

print("\nCole-Cole Interpretation:")
print("- Semicircle → Single relaxation time (Maxwell element)")
print("- Deviation at high ω → Newtonian solvent contribution (η∞·ω)")
print("- Multiple relaxation times would produce skewed/multiple arcs")

## 8. Save Results

In [ ]:
# Create output directory
output_dir = Path("../outputs/dmt/saos")
output_dir.mkdir(parents=True, exist_ok=True)

# Save NLSQ results
nlsq_results = {
    'omega': omega,
    'G_prime_data': G_prime_noisy,
    'G_double_prime_data': G_double_prime_noisy,
    'G_prime_fit': G_prime_fit,
    'G_double_prime_fit': G_double_prime_fit,
    'fitted_params': fitted_params,
    'true_params': {k: calib_params[k] for k in ['G0', 'eta_0', 'eta_inf']}
}

np.savez(output_dir / 'nlsq_results.npz', **nlsq_results)

# Save Bayesian results
bayes_results = {
    'posterior_samples': bayes_result.posterior_samples,
    'G_prime_median': G_prime_median,
    'G_double_prime_median': G_double_prime_median,
    'G_prime_ci_lower': G_prime_lower,
    'G_prime_ci_upper': G_prime_upper,
    'G_double_prime_ci_lower': G_double_prime_lower,
    'G_double_prime_ci_upper': G_double_prime_upper
}

np.savez(output_dir / 'bayesian_results.npz', **bayes_results)

print(f"Results saved to {output_dir}/")
print("  - nlsq_results.npz")
print("  - bayesian_results.npz")

## 9. Key Takeaways

### Theory
1. **Maxwell Equivalence**: At fixed structure ($\lambda_0$), DMT with elasticity is a single Maxwell element
2. **SAOS Limitations**: Linear oscillatory tests only probe $G$, $\eta$, $\eta_\infty$ — thixotropic parameters are invisible
3. **Crossover Frequency**: $\omega_c \approx G/\eta$ provides the characteristic relaxation time

### Identifiability
4. **Only 3 Parameters**: $G_0$, $\eta_0$, $\eta_\infty$ can be determined from SAOS at fixed $\lambda_0$
5. **Thixotropic Degeneracy**: Different $(a, c, t_{\text{eq}}, m_G)$ combinations give identical SAOS
6. **Transient Tests Required**: Need startup, creep, or step-strain to probe structure evolution

### Practical
7. **Preshear Matters**: SAOS depends on sample history ($\lambda_0$)
8. **Cole-Cole Diagnostic**: Semicircle confirms single relaxation mode
9. **Bayesian Uncertainty**: Credible intervals show parameter correlations ($\eta_0$ and $G_0$ trade off via $\theta_1$)

### Next Steps
- **Notebook 06**: LAOS (Large Amplitude) to probe nonlinear structure evolution during oscillation
- **Combined Fitting**: Use SAOS + Startup + Flow Curve → Fully constrain all 7 DMT parameters

## Further Reading

### DMT Model Documentation

- [DMT Overview](../../docs/source/models/dmt/index.rst) — Model hierarchy and selection guide
- [SAOS Protocol Equations](../../docs/source/models/dmt/dmt.rst#saos-small-amplitude-oscillatory-shear) — Linear viscoelastic derivation

### Key References

1. **de Souza Mendes, P. R. (2009).** "Modeling the thixotropic behavior of structured fluids." *J. Non-Newtonian Fluid Mech.*, 164, 66-75.

2. **Thompson, R. L., & de Souza Mendes, P. R. (2014).** "Thixotropic behavior of elasto-viscoplastic materials." *Physics of Fluids*, 26, 023101.

3. **Mewis, J., & Wagner, N. J. (2009).** "Thixotropy." *Advances in Colloid and Interface Science*, 147-148, 214-227. — Review of structure-dependent rheology

4. **Larson, R. G., & Wei, Y. (2019).** "A review of thixotropy and its rheological modeling." *J. Rheology*, 63, 477-501.

5. **Hyun, K., Wilhelm, M., Klein, C. O., et al. (2011).** "A review of nonlinear oscillatory shear tests: Analysis and application of large amplitude oscillatory shear (LAOS)." *J. Rheology*, 55, 1-44. — Transition from SAOS to LAOS